In [ ]:
from bs4 import BeautifulSoup
import sys
import glob
import numpy as np
import pandas as pd

stats = []

pages = np.load(".\\tables-rotten\\pages.npy")

df = pd.DataFrame(data=pages)
df = df.drop_duplicates(subset=0, keep="last")
pages = df.values

DICT = pd.DataFrame(columns = ["url" , "template"])

cc=0
for p in pages:
    with open(p[2], 'r' , encoding="UTF-8") as f:
        webpage = f.read()
        soup = BeautifulSoup(webpage)      
        tags_occurr = []
        for tag in soup.findAll():
            tags_occurr.append(tag.name)        
        tags = pd.DataFrame(data=tags_occurr)
        tags = tags.groupby([0])
        grouped_tags = tags.size()
        grouped_tags
        norm = grouped_tags.div(grouped_tags.sum(axis=0), axis=0)
        DICT.loc[cc] = np.zeros(DICT.shape[1])
        DICT.loc[cc , "url"] = p[0]
        DICT.loc[cc , "template"] = p[1]
        for i in norm.index:
            if i in DICT.columns:
                DICT.loc[cc , i] = norm[i]
            else:
                DICT.insert(2, i, 0)
                DICT.loc[cc , i] = norm[i]
    cc += 1
    print(cc)
    if cc > 10 :
        break

In [ ]:
DICT